In [44]:
import pandas as pd 
import numpy as np
import pickle
from collections import defaultdict

In [2]:
chembl_less_1024_df = pd.read_csv('chembl_less_1024_instances.csv')

In [24]:
assays = list(chembl_less_1024_df.columns)[1:]
print(len(assays))

1046


In [27]:
assays_to_drop = []
for assay in assays:
    if chembl_less_1024_df[assay].count() < 128:
        assays_to_drop.append(assay)

In [29]:
print(len(assays_to_drop))
print(len(chembl_less_1024_df.columns) - 1 - len(assays_to_drop))

394
652


In [30]:
pruned_tasks_df = chembl_less_1024_df.drop(assays_to_drop, axis = 1)

In [31]:
tasks_rem = pruned_tasks_df.columns.tolist()[1:]
print(len(tasks_rem))

652


In [32]:
pruned_rows_df = pruned_tasks_df.dropna(subset=tasks_rem, how='all')

In [33]:
len(pruned_rows_df)

63327

In [14]:
# Now check to ensure the remaining tasks have assay metadata

In [15]:
with open('chembl_assay_name_to_type.pickle', 'rb') as handle:
    chembl_assay_name_to_type = pickle.load(handle)
with open('chembl_assay_type_to_names.pickle', 'rb') as handle:
    chembl_assay_type_to_names = pickle.load(handle)

In [35]:
assays_to_drop = []
for assay in tasks_rem:
    if assay not in chembl_assay_name_to_type:
        assays_to_drop.append(assay)
print('{} Assays with no metadata'.format(len(assays_to_drop)))

7 Assays with no metadata


In [36]:
pruned_tasks_df = pruned_rows_df.drop(assays_to_drop, axis = 1)

In [37]:
len(pruned_tasks_df.columns)

646

In [38]:
# Drop any empty rows 
tasks_rem = pruned_tasks_df.columns.tolist()[1:]
pruned_rows_df = pruned_tasks_df.dropna(subset=tasks_rem, how = 'all')

In [40]:
print(len(pruned_rows_df))
print(len(tasks_rem))

61865
645


In [42]:
filename = 'chembl_less_1024_more_128_645_tasks.csv'
pruned_rows_df.to_csv(filename, index=False)

In [43]:
with open('chembl_1024_tasks.pickle', 'wb') as handle:
    pickle.dump(tasks_rem, handle)

In [45]:
chembl_1024_assay_name_to_type = {}
chembl_1024_assay_type_to_names = defaultdict(list)

for assay in tasks_rem:
    assay_type = chembl_assay_name_to_type[assay]
    chembl_1024_assay_name_to_type[assay] = assay_type
    chembl_1024_assay_type_to_names[assay_type].append(assay)

In [49]:
with open('chembl_1024_assay_name_to_type.pickle', 'wb') as handle:
    pickle.dump(chembl_1024_assay_name_to_type, handle)
with open('chembl_1024_assay_type_to_names.pickle', 'wb') as handle:
    pickle.dump(chembl_1024_assay_type_to_names, handle)